In [17]:
import tarfile
import shutil
import os
import cv2
import pathlib
import numpy as np
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.resnet import ResNet101

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

In [18]:
random_seed = 42
keras.utils.set_random_seed(random_seed)

In [19]:
train_data_dir = pathlib.Path('/kaggle/input/foodx-251/clean_train80_sampling_degradato_diviso/clean_train80_sampling_degradato_diviso')
shuffle_value = True
validation_split = 0.2
batch_size = 256
seed_train_validation = 42

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    validation_split=validation_split,
    seed=seed_train_validation,
    subset="training",
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    validation_split=validation_split,
    seed=seed_train_validation,
    subset="validation",
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

test_data_dir = pathlib.Path('/kaggle/input/foodx-251/val_set_diviso/val_set_diviso')
shuffle_value = True

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

test_degraded_data_dir = pathlib.Path('/kaggle/input/foodx-251/val_set_degraded_diviso/val_set_degraded_diviso')
shuffle_value = True
test_degraded_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_degraded_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 150600 files belonging to 251 classes.
Using 120480 files for training.
Found 150600 files belonging to 251 classes.
Using 30120 files for validation.
Found 11993 files belonging to 251 classes.
Found 11993 files belonging to 251 classes.


In [20]:
base_eff_model = EfficientNetB0(weights="imagenet", include_top=True, input_shape=(224,224,3))
base_eff_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_2 (Rescaling)     (None, 224, 224, 3)          0         ['input_2[0][0]']             
                                                                                                  
 normalization_1 (Normaliza  (None, 224, 224, 3)          7         ['rescaling_2[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_3 (Rescaling)     (None, 224, 224, 3)          0         ['normalization_1

In [22]:
trainable_layer = False
for layer in base_eff_model.layers:
    if layer.name == "block7a_expand_conv":
        trainable_layer=True
    layer.trainable=trainable_layer

eff_model = Model(inputs=base_eff_model.input, outputs=base_eff_model.get_layer('top_dropout').output)
last_layer = base_eff_model.get_layer('top_dropout').output
flatten_layer = Flatten()(last_layer)
classification_layer = Dense(251, activation='softmax')(flatten_layer)
eff_model = Model(inputs=eff_model.input, outputs=classification_layer)

In [23]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
eff_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])
history = eff_model.fit(train_ds, epochs=20, verbose=1, validation_data=val_ds)

Epoch 1/20


2024-01-06 11:06:39.753201: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_5/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


471/471 [==============================] - 317s 654ms/step - loss: 4.8476 - accuracy: 0.1111 - top_k_categorical_accuracy: 0.1968 - val_loss: 4.0150 - val_accuracy: 0.2405 - val_top_k_categorical_accuracy: 0.3863
Epoch 2/20
471/471 [==============================] - 308s 651ms/step - loss: 3.7865 - accuracy: 0.2717 - top_k_categorical_accuracy: 0.4219 - val_loss: 3.5735 - val_accuracy: 0.3145 - val_top_k_categorical_accuracy: 0.4711
Epoch 3/20
471/471 [==============================] - 309s 654ms/step - loss: 3.4682 - accuracy: 0.3285 - top_k_categorical_accuracy: 0.4835 - val_loss: 3.3946 - val_accuracy: 0.3458 - val_top_k_categorical_accuracy: 0.5052
Epoch 4/20
471/471 [==============================] - 308s 652ms/step - loss: 3.2933 - accuracy: 0.3609 - top_k_categorical_accuracy: 0.5170 - val_loss: 3.2889 - val_accuracy: 0.3658 - val_top_k_categorical_accuracy: 0.5247
Epoch 5/20
471/471 [==============================] - 309s 653ms/step - loss: 3.1645 - accuracy: 0.3827 - top_k_cat

In [24]:
eff_model.evaluate(test_ds, batch_size=256)
eff_model.evaluate(test_degraded_ds, batch_size=256)

47/47 [==============================] - 24s 466ms/step - loss: 2.5638 - accuracy: 0.4427 - top_k_categorical_accuracy: 0.6217


[2.5638208389282227, 0.4426749050617218, 0.6216959953308105]

In [25]:
eff_model.save('efficent_degradated_20ep_256batch_sgd_dropout.keras')